In [2]:
%matplotlib inline 
import astropy.io.fits as fits
from astropy.coordinates import SkyCoord
import astropy.io.fits as fits 
import astropy.units as u
import numpy as np
# you can download get_cubeinfo.py from my github
# https://github.com/yzhenggit/yzGALFAHI/blob/master/get_cubeinfo.py
from yzGALFAHI.get_cubeinfo import get_cubeinfo 

In [3]:
def extract_HI_from_cube(tar_ra, tar_dec, header, cube_data, radius_deg):
    '''
    tar_ra/tar_dec: the ra/dec for the sightline, in degree
    radius_deg: the radius of the extraction area, in unit of degree
          Note that, HI4PI's beamsize is 16 arcmin
          and GALFA-HI's beamsize is 4 arcmin. 
    '''
    
    # get sky coordinate for the target
    tar_coord = SkyCoord(ra=tar_ra*u.deg, dec=tar_dec*u.deg, frame='icrs')
    print('Sightline: RA=%.2f, DEC=%.2f, l=%.2f, b=%.2f (degree)'%(tar_ra, tar_dec, 
                                                                   tar_coord.galactic.l.degree, 
                                                                   tar_coord.galactic.b.degree))
    print('Extracted within radius: %.2f arcmin (%.2f deg)'%(radius_deg*60, radius_deg))
    
    # parse the cube header information to get RA/DEC coordinators 
    cube_ra, cube_dec, cube_vel = get_cubeinfo(header)
    cube_coord = SkyCoord(ra=cube_ra*u.deg, dec=cube_dec*u.deg, frame='icrs')
    print('Cube RA range: [%.2f, %.2f], DEC range: [%.2f, %.2f]'%(cube_ra[0, -1], 
                                                                  cube_ra[0, 0], 
                                                                  cube_dec[0, 0], 
                                                                  cube_dec[-1, 0]))
    
    # calculate the distance between the sightline and the whole cube
    dist_coord = tar_coord.separation(cube_coord)
    dist_deg = dist_coord.degree # distance in degree 
    
    # this create a 2d mask of [Dec, RA]
    within_r_2d = dist_deg<=radius_deg/2. # beam should be in unit of degree. 
    print('2D MASK data shape: ', within_r_2d.shape)
    
    # this creates a 3 mask of (Vhel, Dec, RA) so that we can use it to take out the spec within the search area 
    within_r_3d = np.asarray([within_r_2d]*cube_vel.size)
    print('3D MASK data shape: ', within_r_3d.shape)

    # now mask the data 
    data = cube_data.copy()
    data[np.logical_not(within_r_3d)] = np.nan

    # take the mean value along the Dec (axis=2) and the RA (axis=1) directions 
    mean_spec = np.nanmean(np.nanmean(data, axis=2), axis=1)
    
    return cube_vel, mean_spec

You can download the HI4PI data from http://cdsarc.u-strasbg.fr/viz-bin/qcat?J/A+A/594/A116. To get the relevant cubes, go to the FTP tab, select CUBES, then EQ2000, then CAR (meaning the cube data are organized in Cartesian projection). 

In [4]:
tar_ra = 150  # deg
tar_dec = 20 # deg
radius_deg = 16/60. # extract mean spectra from one beam, in unit of degree 

cube_file = '/Volumes/YongData2TB/HI4PI/CAR_F08.fits'
header = fits.getheader(cube_file)
cube_data = fits.getdata(cube_file)